<a href="https://colab.research.google.com/github/maximsachs/AlienVault_office365_Phishing_analysis/blob/master/phishtank_deliverable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://data.phishtank.com/data/online-valid.csv

--2020-10-09 10:42:20--  http://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 104.16.101.75, 104.17.177.85, 2606:4700::6811:b155, ...
Connecting to data.phishtank.com (data.phishtank.com)|104.16.101.75|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.phishtank.com/data/online-valid.csv [following]
--2020-10-09 10:42:20--  https://data.phishtank.com/data/online-valid.csv
Connecting to data.phishtank.com (data.phishtank.com)|104.16.101.75|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1750zhbc38ec0.cloudfront.net/datadumps/verified_online.csv?Expires=1602240150&Signature=QKVnBQmiWnAqZuFkwjiajeowRMsuebe-97rsyimzWN1vC03Bha12tepRsb2aw5QHQZNC1QOOFrcm5tRDB5R2QuTMyUOi7oe9ceWWimMBc1PiYkjMcregL2JWOHSd1tlEobxEjRBo1vBuqqgtNJuJwQkiQMERzo9S2m6EOkFsfNl50vDWFEVcX8WAkRmxU0Bpbag9ad7BMmYpwRDh4hSbPfurgnawmIk6k8~NTLfVvklyCNyqwQnvPuYE97pfhlznyyxacjcVLWVHs1~VjhaBoRJRD

In [10]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pprint
import urllib.request

In [3]:
online_valid_df = pd.read_csv("online-valid.csv")
print(online_valid_df.shape[0], "rows")
online_valid_df.head(10)

14651 rows


,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6802172,https://vps5640.inmotionhosting.com/~ergink5/k...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:52:34+00:00,yes,2020-10-09T10:02:04+00:00,yes,Other
1,6802170,https://nice.constantcontactsites.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:52:33+00:00,yes,2020-10-09T09:59:52+00:00,yes,Other
2,6802163,https://n3plvwcpnl375199.prod.ams3.secureserve...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:30:37+00:00,yes,2020-10-09T09:37:12+00:00,yes,Other
3,6802157,https://n3plvwcpnl375199.prod.ams3.secureserve...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:30:18+00:00,yes,2020-10-09T09:32:46+00:00,yes,Other
4,6802148,https://direct-smbc-co-jp.madeulookads.com/pc/...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:19:26+00:00,yes,2020-10-09T09:24:02+00:00,yes,Other
5,6802144,http://normativa-sicurezza-info.org/BPM/WEBHT/...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:07:59+00:00,yes,2020-10-09T09:11:32+00:00,yes,Other
6,6802143,http://normativa-sicurezza-info.org/BPM/,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:07:54+00:00,yes,2020-10-09T09:11:32+00:00,yes,Other
7,6802134,https://www.configuradoryoigoempresas.com/covi...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:00:28+00:00,yes,2020-10-09T09:09:18+00:00,yes,Other
8,6802133,https://www.karpatenwolle.de/wp-content/plugin...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:00:24+00:00,yes,2020-10-09T09:08:33+00:00,yes,Other
9,6802132,https://adsennials.com/covidapprove/Attachment...,http://www.phishtank.com/phish_detail.php?phis...,2020-10-09T09:00:20+00:00,yes,2020-10-09T09:07:02+00:00,yes,Other




---

# Analysing http vs https


In [4]:
protocol_count = defaultdict(lambda: 0)
for index, row in online_valid_df.iterrows():
  # Checking if first 5 elements of the url match https
  if "https" in row["url"][:6]:
    protocol_count["https"] += 1
  # else checking if its just http:
  elif "http:" in row["url"][:6]:
    protocol_count["http"] += 1
  # otherwise its some random other thing.
  else:
    protocol = row["url"].split(":")[0]
    protocol_count[protocol] += 1

pprint.pprint(dict(protocol_count))

{'http': 7186, 'https': 7465}




---

# Analysing top level domains

In [37]:
tld_count = defaultdict(lambda: 0)
for index, row in online_valid_df.iterrows():
  # Extracting the tld from the url
  tld = row["url"].replace("https://","").replace("http://","").split("/")[0].split(".")[-1]
  tld_count[tld] += 1

tld_df = pd.Series(dict(tld_count))
tld_df.sort_values(ascending=False, inplace=True)

show_top_n = 20

tld_print = tld_df.iloc[:show_top_n]
tld_print["OTHERS"] = tld_df.iloc[show_top_n:].sum()
print(tld_print.to_frame(name="TLD Count").transpose().to_string())
print(f"Percentage it top {show_top_n} tlds: {np.round(100*tld_df.iloc[:show_top_n].sum()/tld_df.sum(), decimals=2)} %")

            com   ru  net  org   br   uk   in   io  xyz   ly  link  top   id  fr  co  de  info  ca  cn  app  OTHERS
TLD Count  8552  754  523  338  279  236  184  123  122  118   107  106  104  95  93  92    90  89  89   89    2468
Percentage it top 20 tlds: 83.15 %




---

# Target Company


In [59]:
# First analysing the targets as they are in the raw dataset.
# Calculating target counts
target_counts = online_valid_df["target"].value_counts()
target_counts["unidentified"] = target_counts.pop("Other")
target_counts.sort_values(inplace=True, ascending=False)
show_top_n = 15
targets_print = target_counts.iloc[:show_top_n]
targets_print["OTHERS"] = target_counts.iloc[show_top_n:].sum()
print(targets_print.to_frame(name="Target Count").to_string())
print(np.round(100*(targets_print["unidentified"]/online_valid_df.shape[0]), decimals=2), "% of targets are unidentified.")

                          Target Count
unidentified                     12206
RuneScape                          477
PayPal                             410
Facebook                           249
eBay, Inc.                         247
Microsoft                          203
Halifax                            119
Amazon.com                          64
Google                              50
Virustotal                          49
Orange                              46
Internal Revenue Service            43
ABSA Bank                           33
Steam                               28
JPMorgan Chase and Co.              24
OTHERS                             403
83.31 % of targets are unidentified.


In [61]:
target_counts.to_frame().T

,unidentified,RuneScape,PayPal,Facebook,"eBay, Inc.",Microsoft,Halifax,Amazon.com,Google,Virustotal,Orange,Internal Revenue Service,ABSA Bank,Steam,JPMorgan Chase and Co.,Twitter,HSBC Group,Three,Adobe,Capitec Bank,Wells Fargo,Allegro,Binance,Caixa,Vodafone,Netflix,Apple,PKO Polish Bank,Development Bank of Singapore,Blockchain,DHL,Banco De Brasil,Yahoo,Bank of America Corporation,Lloyds Bank,American Express,Her Majesty's Revenue and Customs,AT&amp;T,"Banco Santander, S.A.",ING Direct,...,VKontakte,Special,TSB,Barclays Bank PLC,Western Union,Alliance Bank,Cox Communications,ArenaNet,Hotmail,Instagram,Poste Italiane,Itau,Branch Banking and Trust Company,Discover Bank,WalMart,WhatsApp,Delta Air Lines,Suncorp,Banco Real,Das kann Bank,Allied Bank Limited,EE,BMO Financial,Salesforce,Visa,CIBC,TD Canada Trust,Nationwide Building Society,Verizon Wireless,UniCredit,Sky Financial,Centurylink,DocuSign,Tesco,LinkedIn,PNC Bank,Mastercard,Rackspace,Cartasi,Royal Bank of Scotland
target,12206,477,410,249,247,203,119,64,50,49,46,43,33,28,24,23,20,19,17,17,17,16,15,14,13,13,11,10,10,10,10,9,9,9,9,8,8,8,7,7,...,3,3,3,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [69]:
# Trying to guess targets from the url:
# Using method described in https://www.sciencedirect.com/science/article/pii/S0167923616300781 to get target url.
enhanced_target_df = online_valid_df.copy()
for index, row in online_valid_df.iterrows():
  if row["target"] != "Other":
    continue
  for target, count in target_counts.items():
    if target in row["url"]:
      enhanced_target_df["target"][index] = target
      print("Setting new target:", row["url"], target)


target_counts = enhanced_target_df["target"].value_counts()
target_counts["unidentified"] = target_counts.pop("Other")
target_counts.sort_values(inplace=True, ascending=False)
show_top_n = 15
targets_print = target_counts.iloc[:show_top_n]
targets_print["OTHERS"] = target_counts.iloc[show_top_n:].sum()
print(targets_print.to_frame(name="Target Count").to_string())
print(np.round(100*(targets_print["unidentified"]/enhanced_target_df.shape[0]), decimals=2), "% of targets are unidentified.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Setting new target: http://therealproductsonline.com/Netflix11/Netflix/signin.php?country=US-United%20States&amp;lang=en Netflix
Setting new target: http://dreamlandresidencia.com/.wp-html.phph/DHLAUTO/dhl.php?rand=13InboxLightaspxn.1774256418 DHL
Setting new target: http://secure-restore.me/PayPal PayPal
Setting new target: https://paypal.securityform-limited.com/PayPal PayPal
Setting new target: https://paypal.securityform-limited.com/PayPal/ PayPal
Setting new target: http://paypal.securityform-limited.com/PayPal PayPal
Setting new target: https://baliwholesalemarket.com/wp-content/secureNetflix/fd7ef78e60408ffbc66c3bb933f9c0ca/login Netflix
Setting new target: https://inversionesjulio.com/login/linkedIn/linkedIn%20(1)/linkedIn%20(1)/LinkedIn.html LinkedIn
Setting new target: https://www.omegacontractor.co.uk/Netflix/Home/login.php Netflix
Setting new target: https://www.paypal.securityform-limited.com/PayPal PayPal
Setting new target: https://vitaminaonline.info/soporte/eBanking-fi